In [1]:
import nltk
import numpy as np
from pyvi import ViTokenizer
import time
import pandas as pd
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Tập data chatbot

In [2]:
file_tranning = ['./dữ liệu chatbot - dự án cuối kì/bạn bè.txt',
'./dữ liệu chatbot - dự án cuối kì/các câu hỏi phức tạp.txt',
'./dữ liệu chatbot - dự án cuối kì/đất nước.txt',
'./dữ liệu chatbot - dự án cuối kì/địa chỉ.txt',
'./dữ liệu chatbot - dự án cuối kì/du lịch.txt',
'./dữ liệu chatbot - dự án cuối kì/gia đình.txt',
'./dữ liệu chatbot - dự án cuối kì/giải trí.txt',
'./dữ liệu chatbot - dự án cuối kì/học tập.txt',
'./dữ liệu chatbot - dự án cuối kì/nghề nghiệp.txt',
'./dữ liệu chatbot - dự án cuối kì/nghỉ lễ.txt',
'./dữ liệu chatbot - dự án cuối kì/người yêu.txt',
'./dữ liệu chatbot - dự án cuối kì/robot.txt',
'./dữ liệu chatbot - dự án cuối kì/shoping.txt',
'./dữ liệu chatbot - dự án cuối kì/tán gẫu.txt',
'./dữ liệu chatbot - dự án cuối kì/tdtu.txt',
'./dữ liệu chatbot - dự án cuối kì/thông tin cá nhân.txt',
'./dữ liệu chatbot - dự án cuối kì/trò chuyện về đi ăn.txt']

In [3]:
question_train = []
answer_train = []
for i in range(len(file_tranning)):
    with open(file_tranning[i], encoding='UTF-8') as f:
        train_lines = f.readlines()
        for line in train_lines:
            tmp = line.split("__eou__")
            question_train.append(tmp[0].strip()) # strip(): Loại bỏ whitespace đầu cuối string
            answer_train.append(tmp[1].strip())

In [4]:
print((question_train[0]))
print((answer_train[0]))

Thích đánh lộn không?
Ngon nhà vô


# Preprocessing

In [5]:
import string

In [6]:
# print(string.punctuation)

In [7]:
def text_process(mess):
    mess = mess.lower()
    nopunc = [char for char in mess if char not in string.punctuation]
    return  ''.join(nopunc)

In [8]:
# Bỏ dấu câu
for i in range(len(question_train)):
    question_train[i] = text_process(question_train[i])

In [9]:
print(question_train[0])
print(answer_train[0])

thích đánh lộn không
Ngon nhà vô


In [10]:
# Word Segmentation 
for i in range(len(question_train)):
    question_train[i] = ViTokenizer.tokenize(question_train[i])

In [11]:
print(question_train[0])
print(answer_train[0])

thích đánh_lộn không
Ngon nhà vô


# Word2vec

In [12]:
from nltk import word_tokenize
sents= []
for i in range(len(question_train)):
    tokens = word_tokenize(question_train[i])
    sents.append(tokens)

In [13]:
# print(sents)

In [14]:
from gensim.models import Word2Vec
modelw2v = Word2Vec(sentences=sents, size=100, window=5, sg=1, min_count=2)

In [15]:
# chuyển 1 câu thành 1 vector
# vector đại diện cho câu theo phương pháp trung bình
np.seterr(divide='ignore', invalid='ignore')

def convertsent2vec(sent):
    sum_ = np.array([0] * 100)
    for word in sent:
        if(word not in modelw2v.wv):
            continue
        vec = modelw2v.wv[word]
        sum_ = sum_ + vec
    return sum_ / len(sent) # Lấy vector của tất cả từ chia trung bình

In [16]:
from scipy import spatial

In [23]:
def get_answer_word2vec(question):
    start = time.time()
    question = ViTokenizer.tokenize(text_process(question))
    question = convertsent2vec(question)
    
    score_cosin = []
    for i in range(len(question_train)):
        score_cosin.append(1 - spatial.distance.cosine(question, convertsent2vec(question_train[i])))
#     print(score_cosin)

    question_pred = max(score_cosin)
    index = score_cosin.index(question_pred)
    end = time.time()
    
    print(f"Time_word2vec: {end - start}s")

    return  question_train[index], answer_train[index]

# TfidfVectorizer

In [18]:
train_set = pd.Series(question_train)

In [19]:
tokenizer = TfidfVectorizer()

In [20]:
train_tfidf = tokenizer.fit_transform(train_set)

In [24]:
def get_answer_tfidf(question):
    start = time.time()
    input_set = pd.Series([ViTokenizer.tokenize(question)])
    new_tfidf = tokenizer.transform(input_set)
    
    X = pd.DataFrame(cosine_similarity(new_tfidf, train_tfidf), columns=train_set.index)
    
    X['input'] = input_set.index
    score = pd.melt(
        X,
        id_vars='input',
        var_name='train',
        value_name='score'
    )
#     print(score)

    column = score["score"]
    max_value = column.idxmax()
    
    end = time.time()
    
    print(f"Time_tfidf: {end - start}s")
    
    return train_set[max_value], answer_train[max_value]

In [28]:
# pip install beautifultable

Note: you may need to restart the kernel to use updated packages.


In [2]:
from beautifultable import BeautifulTable
import warnings

In [3]:
warnings.filterwarnings("ignore")

In [36]:
while True:
    question_input = input("Q: ")
    if(question_input.strip() == "stop"):
        break
    question_pred_word2vec, answer_word2vec = get_answer_word2vec(question_input)
#     print("Q_pred_word2vec: ", question_pred_word2vec)
    
#     print("A_word2vec: ", answer_word2vec)
    
    question_pred_tfidf, answer_tfidf = get_answer_tfidf(question_input)
    
#     print("Q_pred_tfidf: ", question_pred_tfidf)
    
#     print("A_tfidf: ", answer_tfidf)
    
    table = BeautifulTable()
    table.column_headers = ["", "Word2Vec", "TF-IDF"]
    table.append_row(['Q_pred',  question_pred_word2vec,  question_pred_tfidf])
    table.append_row(['A', answer_word2vec, answer_tfidf])
    print(table)

Q: 1 + 1
Time_word2vec: 0.385998010635376s
Time_tfidf: 0.005990505218505859s
+--------+------------------------------+----------------------+
|        |           Word2Vec           |        TF-IDF        |
+--------+------------------------------+----------------------+
| Q_pred |             1 1              | thích đánh_lộn không |
+--------+------------------------------+----------------------+
|   A    | 1 + 1 còn hỏi thì mày ngu :) |     Ngon nhà vô      |
+--------+------------------------------+----------------------+
---------------------------------------------------------------
Q: bạn có người yêu chưa
Time_word2vec: 0.3840053081512451s
Time_tfidf: 0.004990577697753906s
+--------+----------------------------------+----------------------------------+
|        |             Word2Vec             |              TF-IDF              |
+--------+----------------------------------+----------------------------------+
| Q_pred |      bạn có người_yêu chưa       |      bạn có người_yêu